In [52]:
import astropy
from astropy.io import fits
from astropy.io import ascii
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import odr
from matplotlib.pyplot import figure

In [53]:
#APOKASC file & file variables
hdul = fits.open(r"/Users/amber3/Desktop/SSI/APOKASC_cat_v6.7.2.fits")
hdu = hdul[1]
A_data = hdu.data
#print(np.size(A_data))
KIC_A = hdu.data['KEPLER_INT'] 
Logg_A = hdu.data['DR16_LOGG_COR']
Teff_A = hdu.data['DR16_TEFF_COR']
FeH_A = hdu.data['DR16_M_H_COR']
R_APO3 = hdu.data['APOKASC3P_RADIUS']

In [54]:
#GAIA file variables
#there are some empty entries in error and radius, so flags are put in as -42 and only pos entries are taken
G_data = ascii.read('/Users/amber3/Desktop/gaia_radii_apokasc3 .dat', delimiter = ',') #data_Splitter = 'KIC') #fill_values = [('-42.0', '', '-42.0')])
KIC_G = G_data['KIC']
R_G = 1.0/G_data['inv_r_gaia_K_MIST']
R_G_err_per = G_data['inv_r_gaia_K_MIST_err']/G_data['inv_r_gaia_K_MIST'] 
R_G_err = R_G_err_per * R_G

In [55]:
#APOLLINAIRE file & file variables (b = background)
hdul_b = fits.open(r"/Users/amber3/Desktop/SSI/Selected_stars_final.fits")
hdu_b = hdul_b[0]
B_data = hdu_b.data
KIC_B = np.array(B_data[:,0,0])

In [56]:
#Intersection between Apollinaire and APOKASC
common_KIC, A_indices, B_indices = np.intersect1d(KIC_A, KIC_B, return_indices=True)

good_indices = np.where((Teff_A[A_indices] > 0) & (R_APO3[A_indices] > 0))
bad_indices = np.where(np.logical_or(Teff_A[A_indices] <= 0, R_APO3[A_indices] <= 0))

B_indices = B_indices[good_indices]
A_indices = A_indices[good_indices]

A_data = A_data[A_indices]
B_data = np.array(B_data[B_indices,:,:])

KIC_B = np.array(B_data[:,0,0])

In [57]:
#intersection between Gaia and (Apollinaire & APOkasc)
common_KIC_G, G_indices, AB_indices = np.intersect1d(KIC_G, KIC_B, return_indices=True)
good_indices = np.where((R_G[G_indices] > 0) & (R_G_err[G_indices] > 0))
G_indices = G_indices[good_indices]
AB_indices = AB_indices[good_indices]

G_data = G_data[G_indices]
A_data = A_data[AB_indices]
B_data = B_data[AB_indices]

In [58]:
#Variables
ln10 = np.log(10)
Teff = A_data['DR16_TEFF_COR']
logTeff = np.log10(Teff)
elogTeff = A_data['DR16_TEFF_COR_ERR']/Teff/ln10
logg = A_data['DR16_LOGG_COR']
elogg = A_data['DR16_LOGG_COR_ERR']
FeH = A_data['DR16_M_H_COR']
eFeH = A_data['Dr16_M_H_COR_ERR']
logAmp1 = np.log10(B_data[:,0,6])
elogAmp1 = B_data[:,1,6]/B_data[:,0,6]/ln10
logAmp2 = np.log10(B_data[:,0,9])
elogAmp2 = B_data[:,1,9]/B_data[:,0,9]/ln10
logFreq1 = np.log10(B_data[:,0,7])
elogFreq1 = B_data[:,1,7]/B_data[:,0,7]/ln10
logFreq2 = np.log10(B_data[:,0,10])
elogFreq2 = B_data[:,1,10]/B_data[:,0,10]/ln10
Evolstage = B_data[:,0,3]

In [59]:
#General function for orthogonal fitting of amps and freqs
def odr_fit(A, X):
    x,y,z = X
    return A[0]+A[1]*x+A[2]*y+A[3]*z

In [60]:
def fittings(x,sx,y,sy):
    c_data = odr.RealData(x, y, sx=sx, sy=sy)
    linear = odr.Model(odr_fit)
    beta0 = [183.7, -48.0, -0.73, -0.52] #
    c_odr = odr.ODR(c_data, linear, beta0=beta0)
    c_output = c_odr.run()
    c_output.pprint()
    #return coefficients, save to variable

In [61]:
def plottings():

IndentationError: expected an indented block (3805463755.py, line 1)

In [62]:
for j in (1,2,3,4):
    if j == 1: #all red giants
        ii = iRG
        print('Giants_Gaia')
    elif j == 2: 
        ii = iRC
    elif j == 3: #3 and 4 could be for matched subsample
        ...
    elif j== 4:
        ...
    else:
        print('how did we get here??')
    x = [logTeff[ii], logg[ii], FeH[ii]]
    sx = [elogTeff[ii], elogg[ii], eFeH[ii]]
    #y = [logAmp1[ii], logFreq1[ii], logAmp2[ii], logFreq2[ii]]
    #sy = [elogAmp1[ii], elogFreq1[ii], elogAmp2[ii], elogFreq2[ii]]
    #fittings(x,sx,y,sy)
    
    y = [logAmp1[ii]]
    sy = [elogAmp1[ii]]
    print('A1_fit:')
    #call varaible coefficients here
    fittings(x,sx,y,sy)
    
    y = [logFreq1[ii]]
    sy = [elogAmp1[ii], elogFreq1[ii], elogAmp2[ii], elogFreq2[ii]]
    fittings(x,sx,y,sy)
    
    y = [logAmp1[ii], logFreq1[ii], logAmp2[ii], logFreq2[ii]]
    sy = [elogAmp1[ii], elogFreq1[ii], elogAmp2[ii], elogFreq2[ii]]
    fittings(x,sx,y,sy)
    
    y = [logAmp1[ii], logFreq1[ii], logAmp2[ii], logFreq2[ii]]
    sy = [elogAmp1[ii], elogFreq1[ii], elogAmp2[ii], elogFreq2[ii]]
    fittings(x,sx,y,sy)
    plottings()

NameError: name 'iRG' is not defined